<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=263af4c870df0c37c9f94fa2088b493b2b5960ae2e45e1b00105be8246af802f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-30 11:17:58
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: -50.10 K (-0.34%)
Current PnL: -18.45 L (-12.09%)
CY Booked + Current PnL: -6.32 L (-4.14%)
-------------------
Total profit:  2.66 L
Total loss:  -21.11 L
-------------------
Total Booked + Current PnL: 20.66 L (16.44%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.32%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.52 L (58.68%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.84%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
59    RELIANCE   1533.0    -6.53    73.0       X-LC     4.59  233672.0   
79  TTKPRESTIG    770.0   106.19    58.0       M-SC     1.27   88825.0   
12   BANKINDIA    149.0   -15.53    75.0       H-MC    17.03  217043.0   
42         ITC    452.0   -36.23    64.0       X-LC     1.27  205624.0   
2   ABBOTINDIA  35195.0    -9.84    40.0       X-MC     2.50   89085.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
59      23306.0   5468.0       -0.42         11.08   2.34  13.68     37.0   
79     -11952.0  12045.0       -2.65        -11.86  13.56   0.09    245.0   
12      37235.0  12111.0        0.25         20.71   5.58  27.45     88.0   
42       5486.0  15401.0       -0.26          2.74   7.49  10.44      4.0   
2       -1593.0  16499.0       -0.03         -1.76  18.52  16.44    101.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
59     4.26        1.62    29.43     XY25      NTT  REFINERIES  
79    -0.99        0.62    16.70    OX40N      NTT    DURABLES  
12     3.07        1.50    61.85       XR      NTT       BANKS  
42     0.36        1.43     8.46      X40      NTT        FMCG  
2     -0.10        0.62    18.06      X40      ATH      PHARMA

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
82  VAIBHAVGBL   521.00    87.33    80.0       H-SC     2.49  162562.0   
17        CAMS  5211.76    -2.95    56.0       X-SC     0.52  110407.0   
19   COALINDIA   497.97     7.46    48.0       L-LC     2.41  142986.0   
50      MASFIN   398.61   -19.15    51.0       H-SC     9.16   92040.0   
44      JIOFIN   387.00    -1.39    54.0       X-LC    10.92  267847.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
82     -20213.0  131285.0       11.87        -11.06  80.76  60.77    125.0   
17       8403.0   35518.0        2.27          8.24  32.17  43.06    122.0   
19      -7474.0   41266.0        1.16         -4.97  28.86  22.46    167.0   
50      -5940.0   27548.0        1.12         -6.06  29.93  22.05    152.0   
44        482.0   64980.0        0.55          0.18  24.26  24.48     48.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
82    -0.15        1.13    45.60       XR      NTT  JEWELLERY  
17     0.24        0.77    26.76     X40N      ATH       MISC  
19    -0.18        0.99    12.21     XY25      ATH     MINING  
50    -0.22        0.64    33.79       XR      ATH    FINANCE  
44     0.01        1.86    55.25     XY24      BTT    FINANCE

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
62        SAIL   228.00    49.16    59.0       M-MC    10.22  235397.0   
79  TTKPRESTIG   770.00   106.19    58.0       M-SC     1.27   88825.0   
54       QUESS   986.00   -46.01    48.0       X-SC    37.97   53622.0   
61       ROUTE  2236.84   -55.56    30.0       H-SC    22.17   73038.0   
84         VBL   671.64   -13.57    63.0       X-LC     8.46  311025.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
62      10435.0  157222.0       -2.74          4.64   66.79   74.53    192.0   
79     -11952.0   12045.0       -2.65        -11.86   13.56    0.09    245.0   
54     -11384.0  162314.0       -2.57        -17.51  302.70  232.18    198.0   
61     -60028.0  152883.0       -1.85        -45.11  209.32   69.78    118.0   
84      -4817.0  117474.0       -1.60         -1.53   37.77   35.67      5.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
62     0.07        1.63    38.39     XY24      BTT     STEEL  
79    -0.99        0.62    16.70    OX40N      NTT  DURABLES  
54    -0.07        0.37     4.59     XY24      NTT      MISC  
61    -0.39        0.51     0.04       SR      ATH        IT  
84    -0.04        2.16    12.12     X40N      ATH      FMCG

In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  Current P/L  \
36    IEX  219.0   -30.39    59.0       H-SC    15.93  198936.0      -1018.0   
87  WIPRO  420.0   -14.70    44.0       M-LC     5.68  149829.0      -1116.0   

     FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  RRR Ind  \
36   98254.0       -1.39         -0.51  49.39  48.63    115.0    -0.01   
87  110574.0       -0.26         -0.74  73.80  72.51     53.0    -0.01   

    CurrAlloc%  Gained% Criteria Strategy Category  
36        1.38    10.98       XR      NTT     MISC  
87        1.04     5.24       XR      NTT       IT

In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
79  TTKPRESTIG   770.0   106.19    58.0       M-SC     1.27   88825.0   
46   KANSAINER   340.0   -67.42    51.0       H-SC     2.82  225054.0   
18        CERA  9475.0   -22.91    50.0       H-SC     2.50  140668.0   
74   TATAELXSI  9161.0   -22.53    53.0       H-MC     6.44  104899.0   
68         SIS   528.0  2020.21    43.0       H-SC     3.22   85565.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
79     -11952.0  12045.0       -2.65        -11.86  13.56   0.09    245.0   
46     -44613.0  80952.0       -1.04        -16.54  35.97  13.47    138.0   
18     -35235.0  77255.0       -0.17        -20.03  54.92  23.89    149.0   
74     -26936.0  69160.0       -0.32        -20.43  65.93  32.03     98.0   
68     -25467.0  49072.0       -1.32        -22.94  57.35  21.26    156.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
79    -0.99        0.62    16.70    OX40N      NTT  DURABLES  
46    -0.55        1.56    14.43     XY24      NTT    PAINTS  
18    -0.46        0.97    21.49    OX40N      NTT  CERAMICS  
74    -0.39        0.73    17.50    OX40N      NTT        IT  
68    -0.52        0.59    15.61    OX40N      NTT      MISC

In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Symbol    FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
12  BANKINDIA  149.0   -15.53    75.0       H-MC    17.03  217043.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%  FTT%   OTT%  CumlRnk  \
12      37235.0  12111.0        0.25         20.71  5.58  27.45     88.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
12     3.07         1.5    61.85       XR      NTT    BANKS

In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol     FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
62       SAIL  228.00    49.16    59.0       M-MC    10.22  235397.0   
57     RECLTD  446.00    47.87    57.0       M-LC     5.17  209055.0   
32   HINDZINC  730.22    27.88    49.0       M-LC    11.32  207974.0   
47   KPIGREEN  731.05    11.71    71.0       H-SC     5.46  132905.0   
80  UNIONBANK  163.00    -8.53    62.0       M-LC     9.69  165490.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
62      10435.0  157222.0       -2.74          4.64  66.79  74.53    192.0   
57       5885.0   36250.0       -1.41          2.90  17.34  20.74     55.0   
32       2898.0  109665.0       -1.14          1.41  52.73  54.89     52.0   
47       7608.0   51315.0       -0.81          6.07  38.61  47.03    141.0   
80      24650.0   19842.0       -0.39         17.50  11.99  31.59     66.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy Category  
62     0.07        1.63    38.39     XY24      BTT    STEEL  
57     0.16        1.45     9.96     XY25      NTT  FINANCE  
32     0.03        1.44    25.85      X5K      ATH   METALS  
47     0.15        0.92    66.41       MH      ATH    POWER  
80     1.24        1.15    48.63     XY24      NTT    BANKS

In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
12  BANKINDIA   149.00   -15.53    75.0       H-MC    17.03  217043.0   
86  WHIRLPOOL  2270.00   -37.63    65.0       M-SC     3.73  129407.0   
47   KPIGREEN   731.05    11.71    71.0       H-SC     5.46  132905.0   
37  INDIAMART  4810.62   -53.15    56.0       H-SC     6.67  130751.0   
32   HINDZINC   730.22    27.88    49.0       M-LC    11.32  207974.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%    OTT%  CumlRnk  \
12      37235.0   12111.0        0.25         20.71   5.58   27.45     88.0   
86      14609.0   79430.0       -0.33         12.73  61.38   81.92    223.0   
47       7608.0   51315.0       -0.81          6.07  38.61   47.03    141.0   
37       7415.0  124213.0       -0.67          6.01  95.00  106.72    119.0   
32       2898.0  109665.0       -1.14          1.41  52.73   54.89     52.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
12     3.07        1.50    61.85       XR      NTT     BANKS  
86     0.18        0.90    53.20       XR      NTT  DURABLES  
47     0.15        0.92    66.41       MH      ATH     POWER  
37     0.06        0.91    30.80       AR      ATH      MISC  
32     0.03        1.44    25.85      X5K      ATH    METALS

In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
75  TATAMOTORS  1065.00   -52.51    19.0       X-LC    20.40  149320.0   
13   BATAINDIA  2096.00   -40.01    30.0       X-SC     4.91   89723.0   
24       DMART  5391.45   -17.56    35.0       X-LC     9.68  100800.0   
45    JSWINFRA   342.00   -20.48    37.0       X-MC     5.24  199496.0   
31  HINDUNILVR  2922.00   -11.98    38.0       X-LC     7.53  247230.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
75    -127203.0  237269.0       -0.04        -46.00  158.90  39.81     54.0   
13     -38947.0   84241.0        0.37        -30.27   93.89  35.20    219.0   
24      -1786.0   28597.0       -0.70         -1.74   28.37  26.13     38.0   
45       -851.0   31361.0       -0.22         -0.42   15.72  15.23    178.0   
31      -9833.0   44971.0       -0.64         -3.83   18.19  13.67     24.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
75    -0.54        1.03     5.24     XY24      NTT      AUTO  
13    -0.46        0.62     2.98      X40      NTT  FOOTWEAR  
24    -0.06        0.70    23.39     X40N      ATH      FMCG  
45    -0.03        1.38    31.69     X40N      NTT    REALTY  
31    -0.22        1.71    15.80     XY25      NTT      FMCG

In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
67  SIEMENS   4671.50    -1.76    51.0       H-LC     1.18  157850.0   
42      ITC    452.00   -36.23    64.0       X-LC     1.27  205624.0   
53     PGHH  17907.65   -30.55    38.0       X-MC     2.30  201165.0   
29  HAVELLS   2069.16    -4.76    53.0       X-MC     2.97  317898.0   
85   VOLTAS   1530.00     1.30    53.0       X-MC     4.22  212565.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
67     -28245.0   75721.0        0.26        -15.18  47.97  25.51     15.0   
42       5486.0   15401.0       -0.26          2.74   7.49  10.44      4.0   
53        345.0   67451.0       -0.31          0.17  33.53  33.76     80.0   
29     -10093.0  116637.0        0.29         -3.08  36.69  32.48     92.0   
85      20823.0   16941.0       -0.12         10.86   7.97  19.69     99.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
67    -0.37        1.09    16.32       AR      ATH  ELECTRICAL  
42     0.36        1.43     8.46      X40      NTT        FMCG  
53     0.01        1.39     5.38      X40      ATH        FMCG  
29    -0.09        2.20     8.00      X40      ATH  ELECTRICAL  
85     1.23        1.47    18.40     XY25      NTT          AC

In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
13  BATAINDIA   2096.00   -40.01    30.0       X-SC     4.91   89723.0   
52    PAGEIND  51769.93   -27.54    40.0       X-MC     6.68   81810.0   
54      QUESS    986.00   -46.01    48.0       X-SC    37.97   53622.0   
21     COLPAL   3767.14    -3.45    49.0       X-MC     7.65  225600.0   
3         ACC   3906.00   -37.20    54.0       X-MC     2.51  186800.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
13     -38947.0   84241.0        0.37        -30.27   93.89   35.20    219.0   
52      -1700.0   21729.0       -1.10         -2.04   26.56   23.98     82.0   
54     -11384.0  162314.0       -2.57        -17.51  302.70  232.18    198.0   
21     -37765.0  151107.0       -0.52        -14.34   66.98   43.04     84.0   
3      -51151.0  203799.0       -0.60        -21.50  109.10   64.15    174.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
13    -0.46        0.62     2.98      X40      NTT  FOOTWEAR  
52    -0.08        0.57     3.67      X40      ATH  APPARELS  
54    -0.07        0.37     4.59     XY24      NTT      MISC  
21    -0.25        1.56     4.74     XY25      ATH      FMCG  
3     -0.25        1.29     4.78     XY24      BTT    CEMENT

In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol       FTT  Dev%_PE  RSI_14 Conviction  Spread%  Current  \
54       QUESS    986.00   -46.01    48.0       X-SC    37.97  53622.0   
52     PAGEIND  51769.93   -27.54    40.0       X-MC     6.68  81810.0   
58      RELAXO   1176.00   -39.49    45.0       X-SC     4.77  82429.0   
2   ABBOTINDIA  35195.00    -9.84    40.0       X-MC     2.50  89085.0   
13   BATAINDIA   2096.00   -40.01    30.0       X-SC     4.91  89723.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
54     -11384.0  162314.0       -2.57        -17.51  302.70  232.18    198.0   
52      -1700.0   21729.0       -1.10         -2.04   26.56   23.98     82.0   
58     -62731.0  138662.0       -0.53        -43.22  168.22   52.31    136.0   
2       -1593.0   16499.0       -0.03         -1.76   18.52   16.44    101.0   
13     -38947.0   84241.0        0.37        -30.27   93.89   35.20    219.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
54    -0.07        0.37     4.59     XY24      NTT      MISC  
52    -0.08        0.57     3.67      X40      ATH  APPARELS  
58    -0.45        0.57    10.34     X40N      NTT  FOOTWEAR  
2     -0.10        0.62    18.06      X40      ATH    PHARMA  
13    -0.46        0.62     2.98      X40      NTT  FOOTWEAR

In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
76    TCS  4389.96   -28.36    52.0       X-LC    10.94  285487.0   
40   INFY  2275.00   -18.16    56.0       X-LC     5.78  319138.0   
42    ITC   452.00   -36.23    64.0       X-LC     1.27  205624.0   
84    VBL   671.64   -13.57    63.0       X-LC     8.46  311025.0   
1     ABB  7934.00   -39.57    54.0       H-LC     5.61  252624.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
76     -60469.0  127156.0       -0.67        -17.48  44.54  19.28      1.0   
40       6250.0  165441.0       -0.80          2.00  51.84  54.87      3.0   
42       5486.0   15401.0       -0.26          2.74   7.49  10.44      4.0   
84      -4817.0  117474.0       -1.60         -1.53  37.77  35.67      5.0   
1       -8995.0  128207.0       -0.49         -3.44  50.75  45.57      7.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
76    -0.48        1.98     5.54      X40      ATH          IT  
40     0.04        2.21    10.45      X40      BTT          IT  
42     0.36        1.43     8.46      X40      NTT        FMCG  
84    -0.04        2.16    12.12     X40N      ATH        FMCG  
1     -0.07        1.75     7.49       AR      NTT  ELECTRICAL

In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
7   ASIANTILES   137.00  6988.89    57.0       L-SC     7.82   77762.0   
20   COFFEEDAY    80.00   -45.67    47.0       L-SC    28.15   80486.0   
50      MASFIN   398.61   -19.15    51.0       H-SC     9.16   92040.0   
86   WHIRLPOOL  2270.00   -37.63    65.0       M-SC     3.73  129407.0   
37   INDIAMART  4810.62   -53.15    56.0       H-SC     6.67  130751.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
7      -16048.0   92661.0       -0.90        -17.11  119.16   81.67    269.0   
20     -33063.0   73113.0       -0.90        -29.12   90.84   35.27    268.0   
50      -5940.0   27548.0        1.12         -6.06   29.93   22.05    152.0   
86      14609.0   79430.0       -0.33         12.73   61.38   81.92    223.0   
37       7415.0  124213.0       -0.67          6.01   95.00  106.72    119.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
7     -0.17        0.54    50.59       XR      NTT  CERAMICS  
20    -0.45        0.56    96.99       XR      NTT    HOTELS  
50    -0.22        0.64    33.79       XR      ATH   FINANCE  
86     0.18        0.90    53.20       XR      NTT  DURABLES  
37     0.06        0.91    30.80       AR      ATH      MISC

In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol      FTT  Dev%_PE  RSI_14 Conviction  Spread%   Current  \
82  VAIBHAVGBL   521.00    87.33    80.0       H-SC     2.49  162562.0   
47    KPIGREEN   731.05    11.71    71.0       H-SC     5.46  132905.0   
86   WHIRLPOOL  2270.00   -37.63    65.0       M-SC     3.73  129407.0   
5     ANGELONE  3033.00    16.33    64.0       X-SC     5.55  206312.0   
69    SONACOMS   806.63   -32.06    71.0       M-SC     7.81   84630.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
82     -20213.0  131285.0       11.87        -11.06  80.76  60.77    125.0   
47       7608.0   51315.0       -0.81          6.07  38.61  47.03    141.0   
86      14609.0   79430.0       -0.33         12.73  61.38  81.92    223.0   
5       15306.0   42397.0        0.17          8.01  20.55  30.21    157.0   
69     -16529.0   56533.0        0.09        -16.34  66.80  39.54    202.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
82    -0.15        1.13    45.60       XR      NTT  JEWELLERY  
47     0.15        0.92    66.41       MH      ATH      POWER  
86     0.18        0.90    53.20       XR      NTT   DURABLES  
5      0.36        1.43    30.57     X40N      NTT    FINANCE  
69    -0.29        0.59    19.34       AR      ATH       AUTO

In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.60
1     25       43.92
2     50       74.89

In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.11
LC    32.30
MC    24.55
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.95
X40      16.21
XY25     12.02
XR       11.56
X40N     10.68
AR        7.83
OX40N     6.52
X200      1.76
X5K       1.44
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.52
X-LC    21.63
X-MC    18.73
M-SC    12.63
M-LC     5.08
H-LC     4.60
X-SC     4.53
H-MC     3.87
M-MC     1.63
L-SC     1.43
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                14.52         -2.86   36.43
IT                  12.44        -17.08   79.88
FINANCE             12.14         -7.96   56.90
MISC                 6.84        -16.50   81.05
ELECTRICAL           5.81         -7.58   45.72
BANKS                5.67         -4.63   58.73
PAINTS               5.49        -16.56   33.93
INSURANCE            3.76         -1.81   39.78
AUTO                 2.75        -44.60  104.96
DURABLES             2.35        -14.36   42.18
FOOTWEAR             2.28        -33.80   87.45
TRAVEL               2.18        -42.01  113.11
CERAMICS             2.15        -29.26   84.04
AC                   2.14         -3.45   15.67
BREWERIES            1.64          1.22   19.29
STEEL                1.63          4.43   66.79
REFINERIES           1.62          9.97    2.34
JEWELLERY            1.46        -52.07  104.91
METALS               1.44          1.39   52.73
ENGINEERING          1.43        -17.21   70.82
REALTY               1.38         -0.43   15.72
CEMENT               1.29        -27.38  109.10
CHEMICALS            1.28        -42.42  158.61
HEALTHCARE           1.17         -7.59   34.60
MINING               0.99         -5.23   28.86
ENTERTAINMENT        0.95        -41.46  125.83
POWER                0.92          5.72   38.61
PHARMA               0.62         -1.79   18.52
APPARELS             0.57         -2.08   26.56
HOTELS               0.56        -41.08   90.84
TEXTILES             0.49        -29.28   66.15

In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3248537.0     22
XR        1219486.0     14
AR        1159268.0      9
X40        816976.0     12
X40N       618831.0      9
OX40N      544199.0      9
XY25       447274.0      8
SR         267157.0      2
X5K        109665.0      1
X200        69701.0      1
MH          51315.0      1

In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        3294600.0     24
M-SC        1223081.0     15
X-MC        1171836.0     14
X-LC        1017970.0     13
X-SC         526624.0      6
M-LC         276331.0      4
H-LC         273629.0      3
H-MC         258026.0      3
L-SC         255206.0      3
M-MC         157222.0      1
L-MC          56618.0      1
L-LC          41266.0      1

In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1154571.0      6
           AR         832390.0      5
           XR         747776.0      7
M-SC       XY24       701713.0      6
X-MC       XY24       573871.0      4
X-LC       X40        449785.0      5
           XY24       302249.0      2
X-MC       X40        282950.0      6
X-SC       X40N       280069.0      4
H-SC       SR         267157.0      2
           OX40N      241391.0      3
H-LC       AR         203928.0      2
X-LC       X40N       191795.0      3
H-MC       XY24       176755.0      1
X-MC       XY25       168048.0      2
L-SC       XR         165774.0      2
X-SC       XY24       162314.0      1
M-MC       XY24       157222.0      1
X-MC       X40N       146967.0      2
M-SC       OX40N      144216.0      4
           XY25       127569.0      1
           XR         126633.0      2
           AR         122950.0      2
M-LC       XR         110574.0      1
           X5K        109665.0      1
L-SC       OX40N       89432.0      1
X-SC       X40         84241.0      1
X-LC       XY25        74141.0      3
H-LC       X200        69701.0      1
H-MC       OX40N       69160.0      1
L-MC       XR          56618.0      1
H-SC       MH          51315.0      1
L-LC       XY25        41266.0      1
M-LC       XY25        36250.0      1
           XY24        19842.0      1
H-MC       XR          12111.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
